Note: if using google colab to run this notebook, you need to use chrome browser.

In [ ]:
!git clone https://github.com/stephanzwicknagl/viasp.git

In [ ]:
!pip install -e viasp -e viasp/backend -e viasp/frontend

# <center>Introduction to viASP</center>


viASP is a visualization and interactive explanation tool for ASP programs and their stable models.

viASP allows you to explore the visualization in a variety of ways:

* Visualize the derivation of stable models step-by-step in a tree view
* Inspect iterations of recursive rules individually
* Show reasons for the derivation of individual symbols with arrows
* Relax the constraints of unsatisfiable programs
* Inspect single models
* Add `#show` statements on the fly
* Search models, signatures and rules.

![Example visualization](../docs/img/sprinkler.png)

To get started, the startup module starts the viASP server:

In [1]:
from viasp.server import startup
app = startup.run()

Starting backend at http://localhost:5050


Defining an encoding.

In [2]:
%%file sprinkler.lp
1{rain;sprinkler}1.
wet :- rain.
wet :- sprinkler.

Overwriting sprinkler.lp


# The ``Control`` proxy

To load a visualization, viASP needs the ASP program and its stable models.

viASP provides a proxy object for clingo `Control`. It connects the clingo solver to the viASP server to relay the data.

In [4]:
from viasp import Control

options = ['0']
programs = ['sprinkler.lp']
ctl = Control(options)
for path in programs:
    ctl.load(path)
ctl.ground([("base", [])])

with ctl.solve(yield_=True) as handle:
    for m in handle:
        print("Answer:\n{}".format(m))
        ctl.viasp.mark(m) # mark the answer set for visualization
    print(handle.get())
ctl.viasp.show() # push the visualization to the browser
app.run()        # run the Dash app

Answer:
rain wet
Answer:
sprinkler wet
SAT
[INFO] (2023-09-15 11:57:21) Set models.
[INFO] (2023-09-15 11:57:21) Reconstructing in progress.
[INFO] (2023-09-15 11:57:21) Drawing in progress.


In [1]:
from viasp.server import startup
app = startup.run()

Starting backend at http://localhost:5050


In [4]:
%%file hamiltonian.lp

node(1..4). start(1).
edge(1,2). edge(2,3). edge(2,4). edge(3,1).
edge(3,4). edge(4,1). edge(4,3).

{ hc(V,U) } :- edge(V,U).
reached(V) :- hc(S,V), start(S).
reached(V) :- reached(U), hc(U,V).
:- node(V), not reached(V).
:- hc(V,U), hc(V,W), U!=W.
% :- hc(U,V), hc(W,V), U!=W.
:- hc(V, U), hc(W, V), U != W. % <- uncomment to make the program inconsistent, comment the line above


Overwriting hamiltonian.lp


# Using a clingo `Application`

In [2]:
from clingo.application import clingo_main, Application
from viasp import Control

class App(Application):

    def main(self, ctl, files):
        ctl = Control(control=ctl, files=files)

        for path in files:
            ctl.load(path)
        ctl.ground([("base", [])])
        with ctl.solve(yield_=True) as handle:
            for m in handle:
                ctl.viasp.mark(m)
            print(handle.get())
            # uncomment these lines to visualize the program with inconsistencies
            if handle.get().unsatisfiable:
                ctl = ctl.viasp.relax_constraints()
        ctl.viasp.show()


if __name__ == "__main__":
    clingo_main(App(), ['0', 'hamiltonian.lp'])
    app.run()

clingo version 5.6.1
Reading from hamiltonian.lp
Solving...
UNSAT
[INFO] (2023-09-15 12:03:00) Set models.
[INFO] (2023-09-15 12:03:00) Reconstructing in progress.
[INFO] (2023-09-15 12:03:00) Program constraints transformed.
[INFO] (2023-09-15 12:03:00) Set models.
[INFO] (2023-09-15 12:03:00) Reconstructing in progress.
[INFO] (2023-09-15 12:03:00) Drawing in progress.
UNSATISFIABLE

Models       : 0
Calls        : 1
Time         : 0.039s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.030s


# 3. The adaptable way: Using the viASP ``api`` independently of Clingo

If you already have a program and its stable models, you can use the viASP python ``api``. You can directly mark the stable models without solving the program again. 

In [1]:
from viasp.server import startup
app = startup.run()

Starting backend at http://localhost:5050


In [2]:
import viasp

viasp.add_program_file('hamiltonian.lp')
viasp.mark_from_string('node(1).node(2).node(3).node(4).edge(1,2).edge(2,3).edge(2,4).edge(3,1).edge(3,4).edge(4,1).edge(4,3).start(1).hc(1,2).hc(2,4).hc(3,1).hc(4,3).reached(2).reached(3).reached(4).reached(1).')
viasp.mark_from_string('node(1).node(2).node(3).node(4).edge(1,2).edge(2,3).edge(2,4).edge(3,1).edge(3,4).edge(4,1).edge(4,3).start(1).hc(1,2).hc(2,3).hc(3,4).hc(4,1).reached(2).reached(3).reached(4).reached(1).')
viasp.show()
viasp.clingraph(viz_encoding='viz_hamiltonian.lp', graphviz_type="digraph")
app.run()


[INFO] (2023-09-15 12:05:50) Set models.
[INFO] (2023-09-15 12:05:50) Reconstructing in progress.
[INFO] (2023-09-15 12:05:50) Drawing in progress.
[INFO] (2023-09-15 12:05:50) Clingraph visualization in progress.


Debugging information is available in the log file `viasp.log`.